In [1]:
import pandas as pd
import requests
import numpy as np
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
!pip install geocoder
import geocoder

In [3]:
!pip install folium
import folium

In [4]:
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup

In [5]:
x= requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text #extracting data from website
data = BeautifulSoup(x, 'lxml')

In [6]:
table_post = data.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df_pc = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_pc.columns = ['PostalCode', 'Borough', 'Neighborhood']
df_pc.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
df_pc['Borough'].replace('Not assigned', np.nan, inplace=True) #Removing Not assigned values
df_pc.dropna(subset=['Borough'], inplace=True)

df_pc.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
df_pc2 = df_pc.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index() #grouping by Borough
df_pc2.columns = ['PostalCode', 'Borough', 'Neighborhood']
df_pc2

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [10]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df_pc2.groupby(['PostalCode', 'Borough'])
df_2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [11]:
df_2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Canada'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G') #Checking the function with M4G

[43.70941386000004, -79.36309957799995]

In [14]:
postal_codes = df_2['PostalCode']   #Obtaining latitude and longitude for all postal codes 
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist()]

In [16]:
df_2.head(5)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude']) #adding following attributes to dataframe
df_2['Latitude'] = df_coords['Latitude']
df_2['Longitude'] = df_coords['Longitude']

In [18]:
df_2.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813


In [20]:
#now we are interested in Scarborough so we requiree its geolocation
address = 'Scarborough,Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The Geograpical Co-ordinates of Scarborough,Washington are {}, {}.'.format(latitude_x, longitude_y))

The Geograpical Co-ordinates of Scarborough,Washington are 43.773077, -79.257774.


In [21]:
#now using foresquare API
ID = '5BMI2KBRVOVKZGSTYRDHFYURQ0304VX3LXJ5VCHRUJJQFBEF' 
SECRET = 'CBDLS3OFA5PXGKCB3J1I2IMJK0CQIQVWAKQUFRDGQWOY03NJ' 
VERSION = '20200629'
LIMIT = 30
print('My credentails are:')
print('CLIENT_ID is: ',ID)
print('CLIENT_SECRET is: ',SECRET)
print('version :',VERSION)

My credentails are:
CLIENT_ID is:  5BMI2KBRVOVKZGSTYRDHFYURQ0304VX3LXJ5VCHRUJJQFBEF
CLIENT_SECRET is:  CBDLS3OFA5PXGKCB3J1I2IMJK0CQIQVWAKQUFRDGQWOY03NJ
version : 20200629


In [24]:
#Extracting data using foresquare API within radius of 600 and importing json file
radius = 600 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    ID, 
    SECRET, 
    VERSION, 
    latitude_x, 
   longitude_y, 
    radius, 
   LIMIT)
results = requests.get(url).json()

In [25]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.columns

Index(['reasons.count', 'reasons.items', 'referralId', 'venue.categories',
       'venue.id', 'venue.location.address', 'venue.location.cc',
       'venue.location.city', 'venue.location.country',
       'venue.location.crossStreet', 'venue.location.distance',
       'venue.location.formattedAddress', 'venue.location.labeledLatLngs',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.neighborhood', 'venue.location.postalCode',
       'venue.location.state', 'venue.name', 'venue.photos.count',
       'venue.photos.groups', 'venue.venuePage.id'],
      dtype='object')

In [26]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng'] #location of interested places with geocoordinates
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Disney Store,"[{'id': '4bf58dd8d48988d1f3941735', 'name': 'T...",43.775537,-79.256833
1,SEPHORA,"[{'id': '4bf58dd8d48988d10c951735', 'name': 'C...",43.775017,-79.258109
2,St. Andrews Fish & Chips,"[{'id': '4edd64a0c7ddd24ca188df1a', 'name': 'F...",43.771865,-79.252645
3,DAVIDsTEA,"[{'id': '4bf58dd8d48988d1dc931735', 'name': 'T...",43.776320,-79.258688
4,Tommy Hilfiger,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",43.776015,-79.257369


In [28]:
#we will define a function to find type of venue in attributes
def getcategory_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
#now we need to find the type of venue in the attributes using call function
nearby_venues['venue.categories'] = nearby_venues.apply(getcategory_type, axis=1)

In [32]:
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
3,DAVIDsTEA,Tea Room,43.776320,-79.258688
4,Tommy Hilfiger,Clothing Store,43.776015,-79.257369
5,American Eagle Outfitters,Clothing Store,43.776012,-79.258334
6,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
7,Hot Topic,Clothing Store,43.775450,-79.257929
8,Shoppers Drug Mart,Pharmacy,43.773305,-79.251662
9,Tim Hortons,Coffee Shop,43.769936,-79.260368


In [34]:
a=pd.Series(nearby_venues.venue.categories) #we need a list of each type
a.value_counts()[:10]

AttributeError: 'DataFrame' object has no attribute 'venue'

In [35]:
#we need to modify the names of columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
3,DAVIDsTEA,Tea Room,43.776320,-79.258688
4,Tommy Hilfiger,Clothing Store,43.776015,-79.257369


In [36]:
#now applying 34
a=pd.Series(nearby_venues.categories) #we need a list of each type
a.value_counts()[:10]

Clothing Store    7
Coffee Shop       4
Restaurant        3
Tea Room          2
Gas Station       2
Sandwich Place    2
Pharmacy          2
Wings Joint       1
Bar               1
Discount Store    1
Name: categories, dtype: int64

In [39]:
#now we need to find places to stay in Scarborough
#for this we will define a function using foursquare API
def getNearbyVenues(names, latitudes, longitudes, radius=600):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            ID, 
            SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
#now we are going to find nearby places using above function
Scarborough_venues = getNearbyVenues(names=df_2['Neighborhood'],
                                   latitudes=df_2['Latitude'],
                                   longitudes=df_2['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [41]:
#these above are all the list of places
#now we need to classify the type of places and its frequency in different neighborhoods as follows


In [42]:
#we need to use one-code encoding
Scarborough_one = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

In [43]:
Scarborough_one['Neighborhood'] = Scarborough_venues['Neighborhood'] #adding this attribute


In [46]:
l = [Scarborough_one.columns[-1]] + list(Scarborough_one.columns[:-1])
Scarborough_one = Scarborough_one[l]
Scarborough_grouped = Scarborough_one.groupby('Neighborhood').mean().reset_index()
Scarborough_one.head(5)

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,...,Train Station,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
num_top_venues = 5
for hood in Scarborough_grouped['Neighborhood']:
    print("   "+hood+"   ")
    temp =Scarborough_grouped[Scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','frequency']
    temp = temp.iloc[1:]
    temp['frequency'] = temp['frequency'].astype(float)
    temp = temp.round({'frequency': 2})
    print(temp.sort_values('frequency', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

   Agincourt   
                  venue  frequency
0         Shopping Mall       0.11
1  Hong Kong Restaurant       0.06
2       Badminton Court       0.06
3       Bubble Tea Shop       0.06
4          Skating Rink       0.06


   Alderwood, Long Branch   
               venue  frequency
0        Pizza Place       0.14
1        Gas Station       0.14
2  Convenience Store       0.14
3     Sandwich Place       0.14
4                Gym       0.14


   Bathurst Manor, Wilson Heights, Downsview North   
                 venue  frequency
0          Coffee Shop       0.09
1                 Bank       0.09
2       Ice Cream Shop       0.05
3  Fried Chicken Joint       0.05
4          Supermarket       0.05


   Bayview Village   
                        venue  frequency
0                        Park       0.50
1                       Trail       0.25
2  Construction & Landscaping       0.25
3                 Yoga Studio       0.00
4                Night Market       0.00


   Bedford Park, La

             venue  frequency
0       Theme Park        0.3
1             Park        0.2
2      Pizza Place        0.1
3  Harbor / Marina        0.1
4      Music Venue        0.1


   High Park, The Junction South   
               venue  frequency
0  Convenience Store       0.18
1     Sandwich Place       0.12
2        Pizza Place       0.12
3               Park       0.06
4   Sushi Restaurant       0.06


   Hillcrest Village   
                                      venue  frequency
0                                      Park        0.4
1                                    Bakery        0.2
2                        Chinese Restaurant        0.2
3  Residential Building (Apartment / Condo)        0.2
4                              Music School        0.0


   Humber Summit   
                        venue  frequency
0  Construction & Landscaping       0.25
1                  Hobby Shop       0.25
2      Furniture / Home Store       0.25
3                 Auto Garage       0.25
4      

                        venue  frequency
0                         Bar       0.50
1  Construction & Landscaping       0.25
2           Fish & Chips Shop       0.25
3                      Museum       0.00
4                   Nightclub       0.00


   Runnymede, Swansea   
         venue  frequency
0         Café       0.08
1  Coffee Shop       0.08
2       Bakery       0.06
3          Pub       0.05
4  Pizza Place       0.05


   Runnymede, The Junction North   
                    venue  frequency
0                 Brewery       0.23
1  Furniture / Home Store       0.15
2        Department Store       0.08
3            Gourmet Shop       0.08
4        Storage Facility       0.08


   Scarborough Village   
                  venue  frequency
0         Train Station       0.17
1        Sandwich Place       0.17
2            Restaurant       0.17
3     Indian Restaurant       0.17
4  Fast Food Restaurant       0.17


   South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beau

Here we got the list of place type in particular location.For example at Agincourt there are more shopping malls compare to restaurants.
Everyone will remember only the first ranker :) So we will make a dataframe which contains highest frequency at particular place as follows:

In [49]:
#we define a function for sorting in descending order
def r(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [51]:
#now calling above function
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Scarborough_grouped['Neighborhood']

for ind in np.arange(Scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = r(Scarborough_grouped.iloc[ind, :], num_top_venues)

In [52]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Shopping Mall,Park,Pool,Bubble Tea Shop,Breakfast Spot,Latin American Restaurant,Supermarket,Sushi Restaurant,Discount Store,Chinese Restaurant
1,"Alderwood, Long Branch",Pizza Place,Convenience Store,Gas Station,Sandwich Place,Pub,Coffee Shop,Gym,Event Space,Falafel Restaurant,Ethiopian Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Park,Gift Shop,Bridal Shop,Sandwich Place,Middle Eastern Restaurant,Mobile Phone Shop,Fried Chicken Joint,Restaurant
3,Bayview Village,Park,Trail,Construction & Landscaping,Farm,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Restaurant,Italian Restaurant,Pharmacy,Pub,Sushi Restaurant,Juice Bar,Liquor Store,Intersection
5,Berczy Park,Coffee Shop,Café,Hotel,Seafood Restaurant,Restaurant,Cocktail Bar,Japanese Restaurant,Bakery,Park,Lounge
6,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Café,Park,Skating Rink,Women's Store,Event Space,Eastern European Restaurant,Electronics Store,Elementary School
7,"Brockton, Parkdale Village, Exhibition Place",Café,Bar,Restaurant,Coffee Shop,Diner,Indian Restaurant,Art Gallery,Tibetan Restaurant,Thrift / Vintage Store,Gift Shop
8,"Business reply mail Processing Centre, South C...",Coffee Shop,Café,Hotel,Bar,American Restaurant,Tea Room,Restaurant,Sandwich Place,Asian Restaurant,Sushi Restaurant
9,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Gym,Grocery Store,Italian Restaurant,Café,Bank,Japanese Restaurant,Park,Donut Shop,Pub


The list of top venues of a particular place are found and is as above 

# THANK YOU